<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/notebooks/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ### Removing Tags 

In [34]:
!pip install tokenizer
!pip install ekphrasis
!pip install wordninja
!pip install emoji
!pip install spacy_udpipe
!pip install language_tool_python
!pip install compound-word-splitter
!pip install pyenchant
!sudo apt-get install enchant
!sudo apt-get install hunspell-it

     |████████████████████████████████| 112kB 5.4MB/s 
     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 184kB 17.5MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp36-none-any.whl size=82844 sha256=949735b5859b4db13243afc5ea9df9c96636e172078794ef42205a79fd251759
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-5.9-cp36-none-any.whl size=46451 sha256=503f8fd001c7969561517b0e9a0ddbb12750dbd14b618507afd13b024af58f50
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built ekphrasis ftfy
     |████████████████████████████████| 542kB 4.1MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp36-none-any.whl size=541553 sha256=1d302dd58cdf1f0b63a33ff5ed2ad69b556064dca23ba2fabfd8a27c452eeec5
  Stored in direct

In [89]:
import pandas as pd
import csv
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tokenizer import *
from nltk.corpus import stopwords

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import wordninja

from nltk.stem import SnowballStemmer

import ast

import emoji
import unicodedata

import gzip

import spacy_udpipe
import language_tool_python

import splitter
import enchant
from itertools import groupby

import sys
import os


In [177]:
# directory name:
input_dir = '/content/drive/My Drive/HLT/dataset_training/'
output_dir = '/content/drive/My Drive/HLT/clean_dataset_training/'

# Spec
pd.set_option("display.max_colwidth", None)

In [178]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [179]:
#tsv_tweets_file = open("/content/drive/My Drive/HLT/dataset/haspeede2_reference_taskAB-tweets.tsv")
#tsv_news_file = open("/content/drive/My Drive/HLT/dataset/haspeede2_reference_taskAB-news.tsv")
tsv_file = open(input_dir+"haspeede2_dev_taskAB.tsv")

#dataset_tweets_raw = pd.read_csv(tsv_tweets_file,sep='\t')
#dataset_tweets_raw.rename(columns={"text ": "text"}, inplace=True)
#dataset_news_raw = pd.read_csv(tsv_news_file,sep='\t')
#dataset_news_raw.rename(columns={"text ": "text"}, inplace=True)
dataset_raw = pd.read_csv(tsv_file,sep='\t')
dataset_raw.rename(columns={"text ": "text"}, inplace=True)

# Preprocessing phase
Following the "Preprocessing" section of https://books.openedition.org/aaccademia/4832?lang=it#tocfrom1n4 we will perform:
*   List item
*   List item



### Extraction of the first feature: length of the comment

In [180]:
def comment_length(text):
    return len(text)

In [181]:
dataset_raw['text_length'] = dataset_raw['text'].apply(comment_length)

Extraction of the second feature: percentage of words written in CAPS-LOCK inside the comment.

In [182]:
def caps_lock_words(text):
    words = text.split()
    count_caps_lock = 0
    number_of_words = len(words)
    
    for word in words:
        if word.isupper() == True:
            count_caps_lock = count_caps_lock + 1
            
    return ((count_caps_lock*100)//number_of_words)

In [183]:
dataset_raw['#C-L words'] = dataset_raw['text'].apply(caps_lock_words)

 ###Removing Tags 

In [184]:
def clean_tag(text):
    return re.sub(
        r'@user', ' ', text)

In [185]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_tag)

### Replace the characters ‘&’, ‘@’ respectively in the letters ‘e’, ‘a’

In [186]:
def replace_e_a(text):
    text = re.sub(r'&', 'e', text)
    return re.sub(r'@', 'a', text)

In [187]:
dataset_raw['text'] = dataset_raw['text'].apply(replace_e_a)

### Conversion of disguised bad words

In [188]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    text = re.sub(r' c[.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[.x*@%#$^]+o ', ' cazzo ', text) 
    text = re.sub(r' ca[.x*@%#$^]+ro ', ' cazzaro ', text) 
    text = re.sub(r' c[.x*@%#$^]+i ', ' cazzi ', text) 
    text = re.sub(r' m[.x*@%#$^]+a ', ' merda ', text) 
    text = re.sub(r' m[.x*@%#$^]+e ', ' merde ', text) 
    text = re.sub(r' c[.x*@%#$^]+ulo ', ' culo ', text) 
    text = re.sub(r' p[.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

In [189]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_disguised_bad_words)

### Hashtag splitting and saving

In [190]:
def find_hashtags(text):
    result = re.findall(r'#\S+', text)
    if result:
        return result
    else:
        return None

In [191]:
dataset_raw['hashtags'] = dataset_raw['text'].apply(find_hashtags)

In [192]:
def split_hashtags(text):
    
    text = ' ' + text + ' '
    result = re.findall(r'#\S+', text)
    
    for word in result:
        new_word = " ".join(splitter.split(word[1:].lower(), 'it_IT'))
        if len(new_word)==0:
            new_word =  word[1:]
                  
        text = text.replace(word, new_word)
        
    return text

In [193]:
dataset_raw['text'] = dataset_raw['text'].apply(split_hashtags)

 ### Removing URL 

In [194]:
def clean_URL(text):
    return re.sub(r'URL', ' ', text)

In [195]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_URL)

### Extraction of the fourth feature: number of ‘?’ or ‘!’ inside the comment.


In [209]:
def esclamations_and_questions(text):
    return text.count('!') + text.count('?')

In [212]:
dataset_raw['#?!'] = dataset_raw['text'].apply(esclamations_and_questions)

### Extraction of the fifth feature: number of ‘.’ or ‘,’ inside the comment.


In [213]:
def esclamations_and_questions(text):
    return text.count(',') + text.count('.')

In [214]:
dataset_raw['#.,'] = dataset_raw['text'].apply(esclamations_and_questions)

### Removal of nearby equal vowels, removal of nearby equal consonants if they are more than 2

In [204]:
correct_words_vowels = ['sciiti,',
 'welcomerefugees',
 'livetweet',
 'desiree',
 'canaan',
 'tweet.',
 'weekend',
 'romafeyenoord',
 'ebree,',
 'greencard',
 'creerà',
 'cooperative.',
 'moschee,',
 'cooperanti',
 'streetart',
 'khalidmasood',
 'tweet',
 'woolfe',
 'cooperazione',
 'coop',
 'seehofer',
 'speech',
 'coffee',
 'scooter',
 'street',
 'veemenza',
 'moschee.',
 'maalox.',
 'book',
 'tweet',
 'facebook:',
 'sociial,',
 'coop,',
 'canaaniti.',
 'europee,',
 'cooperative',
 'google',
 'creeranno',
 'mediterranee',
 'cooperazione',
 'cooperativa',
 '“boom”',
 'refugees',
 'moonlight',
 'imaam',
 'shooting',
 'sciiti',
 'sunniti',
 'book',
 'atee.',
 'looking',
 'week',
 'ayaan',
 'temporanee.',
 'idee.',
 'sibiliini',
 'food',
 'refugees',
 'retweeted',
 'boom',
 'keep',
 'vodoo',
 'hooligans',
 'ebree',
 'refugees',
 'speed',
 'bloomberg',
 'riina',
 'hatespeech',
 'google',
 'masood',
 'linee.',
 'boom']

In [206]:
def delete_duplicate_vowels_and_redundant_consonant (text):
    parole = text.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]    
        vocali = ['a','e','i','o','u','y']
        
        for idx,val in enumerate(a):
            if idx == 0:
                stringa += a[idx][0] 
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola.lower() not in correct_words_vowels):
                if len(a[idx])>1: 
                  print(parola)
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
        
    return(stringa)  

In [207]:
dataset_raw['text'] = dataset_raw['text'].apply(delete_duplicate_vowels_and_redundant_consonant)

### Punctuation removal

In [ ]:
def clean_punctuation(text):
    text = ' ' + text + ' '
    text = re.sub(r'\\n', '. ', text)
    text = re.sub(r'\\', ' ', text)
    text = re.sub(r'/', ' ', text)
    return re.sub(r'_', ' ', text)

In [215]:
dataset_raw

,id,text,hs,stereotype,text_length,#C-L words,hashtags,#?!,"#.,"
0,2066,"È terrorismo anche questo, per mettere in uno stato di soggezione le persone e renderle innocue, mentre qualcuno.",0,0,120,10,None,0,3
1,2045,infatti finché ci hanno guadagnato con i campi rom tutto era ok con alemanno ipocriti,0,0,101,0,"[#rom, #Alemanno, #Ipocriti]",0,0
2,61,"Corriere: Tangenti, Mafia Capitale dimenticataMazzette su buche e campi rom rom a",0,0,86,8,[#roma],0,1
3,1259,"ad uno ad uno, perché quando i migranti israeliti arrivarono in terra di Canan fecero fuori tutti i Cananiti.",0,0,118,0,None,0,2
4,949,Il divertimento del giorno? Trovare i patrioti italiani che inneggiano contro i rom facendo la spesa alla li dl (multinazionale tedesca).,0,0,138,0,[#Lidl],1,1
...,...,...,...,...,...,...,...,...,...
6832,9340,"Gli stati nazionali devono essere pronti a rinunciare alla propria sovranità. Lo ha detto la Merkel , che ha aggiunto che gli stati nazionali non devono ascoltare la volontà dei loro cittadini quando si tratta di questioni che riguardano immigrazione, confini, o persino sovranità",0,0,285,2,None,0,4
6833,9121,"Il ministro dell'interno della Germania HorstSehofer,sta facendo la proposta di dare soldi agli immigrati che vogliono tornare a casa e aiutarli a creare un'attività a casa loro e fare busines con la Germania.Chi paga?Una parte i crucchi e il resto l'Europa, cioè io e voi!",0,0,277,0,"[#HorstSeehofer,sta]",2,3
6834,8549,"Salvini: In Italia troppi si sono montati la testa, io ringrazio Dio e voi per questi mesi straordinari. Vi raccontavano che su immigrazione non si poteva fare nulla, è bastato usare buonsenso e coraggio. io ci sono piazza del popolo",0,0,233,0,"[#Salvini:, #iocisono, #piazzadelpopolo]",0,4
6835,9240,Chi giubila in buona fede non ha capito niente. Purtroppo credo che i più non siano in buona fede. I migranti sono un grosso busines e chi finora li ha voluti non vuole perdere questo guadagno,0,0,206,2,None,0,2
